In [1]:
import pandas as df
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange

import os
import sys
from pathlib import Path

os.chdir(Path(sys.path[0]).parent)
import modules.utils as utils

import multiprocessing as mp
import time

In [2]:
def random_square(seed):
    np.random.seed(seed)
    random_num = np.random.randint(0, 10)
    return random_num**2

In [3]:
t0 = time.time()
results = []
for i in trange(1000000): 
    results.append(random_square(i))
t1 = time.time()
print(f'Execution time {t1 - t0} s')

  0%|          | 0/1000000 [00:00<?, ?it/s]

Execution time 5.559435844421387 s


In [5]:
t0 = time.time()
n_cpu = mp.cpu_count()

pool = mp.Pool(processes=n_cpu)
results = [pool.map(random_square, trange(1000000))]
t1 = time.time()
print(f'Execution time {t1 - t0} s')

  0%|          | 0/1000000 [00:00<?, ?it/s]

Process SpawnPoolWorker-41:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/queues.py", line 368, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'random_square' on <module '__main__' (built-in)>
Process SpawnPoolWorker-42:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework

KeyboardInterrupt: 

In [6]:
Directory = "data"
L = 32

u_ds = utils.concatenate_time(Directory, 'u')
v_ds = utils.concatenate_time(Directory, 'v')
w_ds = utils.concatenate_time(Directory, 'w')
theta_ds = utils.concatenate_time(Directory, 'theta')
assert u_ds.shape == v_ds.shape == w_ds.shape == theta_ds.shape, 'u,v,w,theta have different shape'

u_coarse = utils.coarse_array(u_ds, L)
v_coarse = utils.coarse_array(v_ds, L)
w_coarse = utils.coarse_array(w_ds, L)

wtheta_ds = w_ds*theta_ds
tke_ds = utils.coarse_array(u_ds*u_ds, L) - u_coarse*u_coarse + utils.coarse_array(v_ds*v_ds, L) - v_coarse*v_coarse + utils.coarse_array(w_ds*w_ds, L) - w_coarse*w_coarse
tke_in = utils.variable_samples(tke_ds, 1)
output_ds = utils.variable_samples(wtheta_ds, L)

variables = ['u', 'v', 'w', 'theta']  # add 's' 
input_ds = utils.input_dataset(Directory, variables, L)
tot_ds = np.concatenate((np.concatenate((input_ds,tke_in), axis=1), output_ds), axis=1)

variables.append('tke')
variables.append('wtheta')
print(variables)

utils.write_nc_file(tot_ds,str(L),variables,4)

['u', 'v', 'w', 'theta', 'tke', 'wtheta']
writing out


In [7]:
path_data = 'data/input_dataset_L_32.nc'
ds_init = xr.open_dataset(path_data)
df_init = ds_init.to_dataframe()
df_init.head

<bound method NDFrame.head of              sample
index i            
0     0   14.980471
      1   14.990431
      2   15.000865
      3   15.009308
      4    0.500632
...             ...
255   19   0.875857
      20   0.132721
      21   0.138177
      22   0.143849
      23   0.149717

[6144 rows x 1 columns]>

In [8]:
nz=4
len_samples = nz*len(variables)
tot_ds = df_init.to_numpy()
n_samples = len(tot_ds)//len_samples
tot_ds = tot_ds.reshape(n_samples, len_samples)
tot_ds.shape

(256, 24)

In [9]:
Directory = "data"
L = 32

var = 'u'

ds = utils.concatenate_time(Directory, var)

coarse_ds = utils.coarse_array(ds, L)

utils.write_coarse_file(coarse_ds,str(L),var,4)

writing out


In [10]:
path_data = 'data/u_L_32.nc'
ds_init = xr.open_dataset(path_data)
df_init = ds_init.to_dataframe()
df_init.head

<bound method NDFrame.head of                       u
time x  y  z           
0.0  0  0  0  14.980471
           1  14.990431
           2  15.000865
           3  15.009308
        1  0  14.464854
...                 ...
     15 14 3  15.269173
        15 0  14.420309
           1  14.436877
           2  14.453506
           3  14.465678

[1024 rows x 1 columns]>